In [15]:
# Imports
import pandas as pd
import gpxpy
import os
from pathlib import Path
from tqdm import tqdm
from route import Route

In [2]:
# Display options
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 100)

In [3]:
# Variables
DATA_PATH = '../data/'
GPX_FILE_PATH = DATA_PATH + 'gpx/'
ROUTES_INDEX_PATH = DATA_PATH + 'routes-index.csv'

In [4]:
test_route = Route(os.path.join(GPX_FILE_PATH, 'hardrock-counterclockwise-2025.gpx'))
test_route.route_name


'Hardrock 100 2023 (Counter-Clockwise)'

In [7]:
gpx_dir = Path(GPX_FILE_PATH)
gpx_files = list(gpx_dir.glob('*.gpx'))
gpx_files[0].name

'goatultratrail-uae-50k-2025.gpx'

In [24]:
# Function definitions
def check_uniques(df, col_name):
    """
    Checks if the values in a DataFrame column are unique. If there are duplicates, prints
    the duplicates.

    Args:
        df (DataFrame): The DataFrame to check.
        col_name (str): The name of the column to check.

    Returns:
        list: A list of the duplicate values.
    """
    duplicates = df[col_name].value_counts()[df[col_name].value_counts() > 1].index.tolist()
    
    if duplicates:
        print(f"Duplicate values in column {col_name}: {duplicates}")
        return duplicates
    else:
        print(f"All values in column {col_name} are unique")
        return []

def process_gpx_files(gpx_file_path=GPX_FILE_PATH):
    """
    Reads all GPX files in the given directory, parsing out their track names and returns
    a DataFrame with the file names and track names. Also checks for duplicates in the
    track names.

    Args:
        gpx_file_path (str): The path to the directory containing the GPX files.

    Returns:
        DataFrame: A DataFrame with the file names and track names.
    """
    # Get list of GPX files
    gpx_dir = Path(gpx_file_path)
    gpx_files = list(gpx_dir.glob('*.gpx'))
    gpx_files.sort()

    # Create empty lists to store data
    file_names = []
    route_names = []
    route_distances = []
    avg_elevations = []

    pbar = tqdm(gpx_files, desc='Processing GPX files', unit='file')
    
    # Process each GPX file
    for gpx_file in pbar:
        pbar.set_description(f'Processing {gpx_file.name}')
        # Create Route object to access metrics
        route = Route(os.path.join(GPX_FILE_PATH, gpx_file.name))
        
        file_names.append(route.file_name)
        route_names.append(route.route_name)
        route_distances.append(route.total_distance)
        avg_elevations.append(route.avg_elevation_gain_per_km)
        
        if route.route_name is None:
            print(f"No route name found for {gpx_file}")

    # Create DataFrame
    df = pd.DataFrame({
        'file_name': file_names,
        'route_name': route_names,
        'route_distance_m': route_distances,
        'avg_elevation_gain_per_km': avg_elevations
    })

    duplicates = check_uniques(df, 'route_name')

    return df

In [25]:
df = process_gpx_files()

df

Processing xterra-ilhagrande-5k-2025.gpx: 100%|██████████| 92/92 [01:26<00:00,  1.06file/s]                                               

All values in column route_name are unique


,file_name,route_name,route_distance_m,avg_elevation_gain_per_km
0,cambotas-cocais-11-2024.gpx,Cambotas Trail Fest 2024 - 11km,11188.007659,45.799039
1,cambotas-cocais-17-2024.gpx,Cambotas Trail Fest 2024 - 17km,16979.093543,51.816665
2,cambotas-cocais-27-2024.gpx,Cambotas Trail Fest 2024 - 27km,26990.645039,51.214041
3,cambotas-cocais-45-2024.gpx,Cambotas Trail Fest 2024 - 45km,44352.361176,50.380632
4,caminhosdecaravaggio-gramado-100-2024.gpx,Ultramaratona Caminhos de Caravaggio 2024 - UMCC 100K,107927.874766,30.268362
...,...,...,...,...
87,xterra-ilhabela-21k-2025.gpx,XTERRA 2025 Ilhabela - Trail Run 21K,20859.438117,33.948326
88,xterra-ilhabela-5k-2025.gpx,XTERRA 2025 Ilhabela - Trail Run 5K,4878.878120,40.813071
89,xterra-ilhagrande-10k-2025.gpx,XTERRA 2025 Ilha Grande - Trail Run 10K,10197.757358,37.666517
90,xterra-ilhagrande-21k-2025.gpx,XTERRA 2025 Ilha Grande - Trail Run 21K,21070.695429,51.468259


In [26]:

# Save to CSV
df.to_csv(ROUTES_INDEX_PATH, index=False)